In [ ]:
!pip install langchain
!pip install openai
!pip install pickle
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os

# Laden Sie die Umgebungsvariablen aus der .env-Datei
load_dotenv()
API_KEY = os.environ.get("API_KEY")

## Loaders  
Um Daten mit einem LLM zu verwenden, müssen Dokumente zunächst in eine Vectordatenbank. 
Der erste Schritt ist diese über einen Loader in memory zu laden 

In [ ]:
from langchain.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader('./FAQ', glob="**/*.txt", loader_cls=TextLoader, show_progress=True)
docs = loader.load()

## Text Splitter
Texte werden nicht 1:1 in die Datenbank geladen, sondern in Stücken, sog. "Chunks". Man kann die Chunk Größe und den Overlap zwischen den Chunks definieren

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)

documents = text_splitter.split_documents(docs)
documents[0]

## Embeddings
Texte werden nicht als Text in der Datenbank gespeichert, sondern als Vectorrepräenstation.
Embeddings sind eine Art von Wortdarstellung, die die semantische Bedeutung von Wörtern in einem Vektorraum darstellt. 

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=API_KEY)


## Laden der Vectoren in VectorDB (FAISS)
Wie von OpenAIEmbeddings erstellen Vectoren können nun in der Datenbank gespeichert. Die DB kann als .pkl file abgelegt werden

In [ ]:
from langchain.vectorstores.faiss import FAISS
import pickle

vectorstore = FAISS.from_documents(documents, embeddings)

with open("vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

## Laden der Datenbank
Vor der Verwendung der Datenbank muss diese natürlich wieder geladen werden. 

In [ ]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

## Prompts
Bei einem LLM hat man die Möglichkeit, diesem vor einer Konversersation eine Identität zu verpassen oder zu definieren wie Frage und Antwort aussehen sollen 

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Du bist ein Tierarzt, Usern beim Umgang mit ihrem Tier .

{context}

Question: {question}
Antwort hier:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


## Chains
Mit Chain Klassen kann man das Verhalten des LLMs leicht beeinflussen

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": PROMPT}

llm = OpenAI(openai_api_key=API_KEY)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), chain_type_kwargs=chain_type_kwargs)

query = "Wie alt wird ein Huninhen?"
qa.run(query)


## Memory
In dem eben gezeigten Beispiel steht jede Anfrage für sich. Eine große Stärke eines LLM ist allerdings, dass diese bei einer Antwort den kompletten Chatverlauf berücksichtigen kann. Dafür muss allerdings aus den unterschiedlichen Fragen und Antworten eine Chathistorie aufgebaut werden. Mit unterschiedlichen Memory Klassen ist dies in Langchain sehr einfach.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer')

## Memory in Chains verwenden
Die Memory Klasse kann nun einfach in einer Chain verwendet werden. Erkennbar ist dies z.B. daran, dass wenn man von "es" spricht, der Bot das Huninchen in diesem Kontext versteht.

In [ ]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(model_name="text-davinci-003", temperature=0.7, openai_api_key=API_KEY),
    memory=memory,
    retriever=vectorstore.as_retriever(),
    combine_docs_chain_kwargs={'prompt': PROMPT}
)


query = "Wie alt wird ein Huninhen?"
qa({"question": query})
qa({"question": "Und wie viel frisst es?"})